<a href="https://colab.research.google.com/github/ttogle918/AI_practice/blob/main/QA%20task/01_BERT_QA_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [12]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# 모델 설명

'bert-large-uncased-whole-word-masking-finetuned-squad' : 스탠포드 질문-응답 데이터셋(SQUAD)을 기반으로 파인튜닝된 모델

In [4]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [33]:
question = "What is the immune system?" #"면역 체계는 무엇입니까?"
paragraph = "The immune system is a system of various biological structures and processes within an organism that protects against disease. To function properly, the immune system must detect a variety of substances known as pathogens, from viruses to parasites, and distinguish them from the healthy tissue of an organism." # "면역 체계는 질병으로부터 보호하는 유기체 내의 다양한 생물학적 구조와 과정의 시스템입니다. 제대로 기능하려면 면역 체계가 바이러스에서 기생충에 이르기까지 병원균으로 알려진 다양한 물질을 탐지하고 유기체의 건강한 조직과 구별해야 합니다."

In [34]:
question = '[CLS]' + question + '[SEP]'
paragraph = paragraph + '[SEP]'

In [35]:
question_tokens = tokenizer.tokenize(question)
paragraph_tokens = tokenizer.tokenize(paragraph)
tokens = question_tokens + paragraph_tokens
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(input_ids)

[101, 2054, 2003, 1996, 11311, 2291, 1029, 102, 1996, 11311, 2291, 2003, 1037, 2291, 1997, 2536, 6897, 5090, 1998, 6194, 2306, 2019, 15923, 2008, 18227, 2114, 4295, 1012, 2000, 3853, 7919, 1010, 1996, 11311, 2291, 2442, 11487, 1037, 3528, 1997, 13978, 2124, 2004, 26835, 2015, 1010, 2013, 18191, 2000, 23996, 1010, 1998, 10782, 2068, 2013, 1996, 7965, 8153, 1997, 2019, 15923, 1012, 102]


In [36]:
segment_ids = [0] * len(question_tokens)
segment_ids += [1] * len(paragraph_tokens)
print(segment_ids)
len(input_ids), len(segment_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


(63, 63)

In [37]:
tokenizer.decode(torch.tensor(input_ids))

'[CLS] what is the immune system? [SEP] the immune system is a system of various biological structures and processes within an organism that protects against disease. to function properly, the immune system must detect a variety of substances known as pathogens, from viruses to parasites, and distinguish them from the healthy tissue of an organism. [SEP]'

In [38]:
input_ids = torch.tensor([input_ids])
segment_ids = torch.tensor([segment_ids])
input_ids

tensor([[  101,  2054,  2003,  1996, 11311,  2291,  1029,   102,  1996, 11311,
          2291,  2003,  1037,  2291,  1997,  2536,  6897,  5090,  1998,  6194,
          2306,  2019, 15923,  2008, 18227,  2114,  4295,  1012,  2000,  3853,
          7919,  1010,  1996, 11311,  2291,  2442, 11487,  1037,  3528,  1997,
         13978,  2124,  2004, 26835,  2015,  1010,  2013, 18191,  2000, 23996,
          1010,  1998, 10782,  2068,  2013,  1996,  7965,  8153,  1997,  2019,
         15923,  1012,   102]])

# 응답 얻기

In [46]:
output = model(input_ids, token_type_ids = segment_ids)
start_scores, end_scores = output['start_logits'], output['end_logits']
start_scores, end_scores

(tensor([[-6.1970, -4.6508, -6.7321, -6.2480, -5.7971, -8.4799, -9.0027, -6.1970,
           2.2423, -0.8495, -4.1347,  2.0661,  6.9412,  2.9878, -1.9175,  2.7158,
           2.6131, -0.8105, -4.6964, -0.7089, -0.7325, -1.3356, -0.8710, -1.3109,
           1.7955, -3.5434, -1.8315, -6.1969, -3.9591, -5.5795, -6.1739, -7.0939,
          -1.1622, -2.8861, -5.6574, -4.0569, -3.5694, -5.4991, -5.7938, -7.2431,
          -5.1451, -6.2245, -6.9027, -3.6589, -6.9411, -8.0326, -5.7052, -4.3933,
          -7.9214, -4.8585, -7.8885, -6.9218, -4.2360, -7.1410, -7.2067, -6.6935,
          -5.0090, -5.1363, -7.5691, -6.9695, -5.9212, -7.7813, -6.1972]],
        grad_fn=<CloneBackward0>),
 tensor([[-9.1324e-01, -4.8485e+00, -6.2943e+00, -7.5126e+00, -6.1771e+00,
          -4.0133e+00, -6.6874e+00, -9.1312e-01, -4.3799e+00, -4.2811e+00,
          -1.0389e+00, -4.2763e+00, -2.1495e+00,  6.5320e-03, -3.0878e+00,
          -3.1975e+00, -9.6219e-01,  1.7029e+00, -4.2877e+00,  3.7762e+00,
          -2.010

In [47]:
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
start_index, end_index

(tensor(12), tensor(26))

In [54]:
print(question[5:-5])
print(' '.join(tokens[start_index:end_index+1]))

# 질병으로부터 보호하는 유기체 내의 다양한 생물학적 구조와 과정의 시스템

What is the immune system?
a system of various biological structures and processes within an organism that protects against disease
